<a href="https://colab.research.google.com/github/ameyutpat/Core-Java-Notes/blob/main/ReviewSummary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [74]:
pip install -U weaviate-client

In [75]:
import os

os.environ["WEAVIATE_URL"] = "https://mdow3hcct0umcgqh2pbzg.c0.asia-southeast1.gcp.weaviate.cloud"
os.environ["WEAVIATE_API_KEY"] = "GTwGSKjjAwYl6Xo5dZLSUXjxoe3ZXC49eRmh"

# Check if it's set
print(os.getenv("WEAVIATE_URL"))  # Output: https://your-cluster-name.weaviate.network
print(os.getenv("WEAVIATE_API_KEY"))

https://mdow3hcct0umcgqh2pbzg.c0.asia-southeast1.gcp.weaviate.cloud
GTwGSKjjAwYl6Xo5dZLSUXjxoe3ZXC49eRmh


In [76]:
import weaviate
from weaviate.classes.init import Auth
import os

# Best practice: store your credentials in environment variables
weaviate_url = os.environ["WEAVIATE_URL"]
weaviate_api_key = os.environ["WEAVIATE_API_KEY"]




In [77]:
'''
client = weaviate.connect_to_weaviate_cloud(
    cluster_url=weaviate_url,
    auth_credentials=Auth.api_key(weaviate_api_key),
)
'''

'\nclient = weaviate.connect_to_weaviate_cloud(\n    cluster_url=weaviate_url,\n    auth_credentials=Auth.api_key(weaviate_api_key),\n)\n'

In [78]:
import pandas as pd

df = pd.read_csv("/content/sample_data/Musical_instruments_reviews.csv",
                usecols = ['reviewerID', 'asin', 'reviewText', 'overall', 'summary', 'reviewTime'])

df = df[df.reviewText.notna()]

In [79]:
os.environ["X-Cohere-Api-Key"] = "w5zpWVezkv4f1MVplkjFTn8sgRfh4bSsxV61xlsw"

print(os.getenv("X-Cohere-Api-Key"))

w5zpWVezkv4f1MVplkjFTn8sgRfh4bSsxV61xlsw


In [34]:
COHERE_API_KEY = os.environ["X-Cohere-Api-Key"]

def create_client():
    """Creates a Weaviate client with Cohere embeddings."""
    client = weaviate.Client(
        embedded_options=EmbeddedOptions(),  # Runs Weaviate embedded
        additional_headers={"X-Cohere-Api-Key": COHERE_API_KEY}
    )
    return client

In [80]:
client = weaviate.connect_to_weaviate_cloud(
    cluster_url=weaviate_url,  # Replace with your Weaviate Cloud URL
    auth_credentials=Auth.api_key(weaviate_api_key),  # Replace with your Weaviate Cloud key
    headers={'X-Cohere-Api-Key': COHERE_API_KEY}  # Replace with your OpenAI API key
)

In [81]:
def create_schema(client):
    """Creates a schema for storing review summaries."""
    class_obj = {
        "class": "Reviews",
         "properties":[
              {
                  "name": "review_text",
                  "dataType": ["text"],
              },
              {
            "name": "product_id",
            "dataType": ["text"],
            "moduleConfig": {
                "text2vec-openai": {
                    "skip": True, # skip vectorization for this property
                    "vectorizePropertyName": False
                }
            }
        }, {
            "name": "reviewer_id",
            "dataType": ["text"],
            "moduleConfig": {
                "text2vec-openai": {
                    "skip": True, # skip vectorization for this property
                    "vectorizePropertyName": False
                }
            }
        },

         ]
    }

    # Ensure schema is not duplicated
    if not client.schema.exists("ReviewSummary"):
        client.schema.create_class(class_obj)


In [82]:
from weaviate.util import generate_uuid5

def batch_reviews(client):
  # Configure batch
  client.batch.configure(batch_size=100)

  # Initialize batch process
  with client.batch as batch:
      for _, row in df.iterrows():
          review_item = {
              "review_text": row.reviewText,
              "product_id": row.asin,
              "reviewer_id": row.reviewerID,
          }

          batch.add_data_object(
              class_name="Reviews",
              data_object=review_item,
              uuid=generate_uuid5(review_item)
          )

In [83]:
generate_prompt = """
Summarize these customer reviews into a one-paragraph long overall review:
{review_text}
"""

In [84]:
def search_reviews(client):
  client.query\
                .get('Reviews',
                     ['review_text', "product_id"])\
                .with_where({
                    "path": ["product_id"],
                    "operator": "Equal",
                    "valueText": product_id
                })\
                .with_generate(grouped_task=generate_prompt)\
                .do()["data"]["Get"]["Reviews"]

In [85]:
def mainFun(client):
  new_review_summary = {
          "product_id" : product_id,
          "summary": summary[0]["_additional"]["generate"]["groupedResult"]
      }

  # Create new object
  client.data_object.create(
    data_object = new_review_summary,
    class_name = "Products",
    uuid = generate_uuid5(new_review_summary)
  )

In [86]:
def repeatMainFun(client):
  generate_prompt = """
  Summarize these customer reviews into a one-paragraph long overall review:
  {review_text}
  """

  for product_id in list(df.asin.unique()):
      # Generate summary
      summary = client.query\
                  .get('Reviews',
                      ['review_text', "product_id"])\
                  .with_where({
                      "path": ["product_id"],
                      "operator": "Equal",
                      "valueText": product_id
                  })\
                  .with_generate(grouped_task=generate_prompt)\
                  .do()["data"]["Get"]["Reviews"]

      new_review_summary = {
          "product_id" : product_id,
          "summary": summary[0]["_additional"]["generate"]["groupedResult"]
      }

      # Create new object
      client.data_object.create(
        data_object = new_review_summary,
        class_name = "Products",
        uuid = generate_uuid5(new_review_summary)
      )

In [87]:
def searachReviewByProduct(client):
  reviews = client.query\
                  .get('Reviews', ['review_text', "product_id"])\
                  .with_where({
                      "path": ["product_id"],
                      "operator": "Equal",
                      "valueText": "1384719342"
                  })\
                .do()
  print(reviews)
  return reviews

In [88]:
def searachReviewByProduct(client):
  res = client.query\
              .get('Products', ['product_id', 'summary'])\
              .with_where({
                  "path": ["product_id"],
                  "operator": "Equal",
                  "valueText": "1384719342"
              })\
              .do()
  print(res)